In [16]:
import gymnasium as gym
import gym_unbalanced_disk, time
import numpy as np
from stable_baselines3 import PPO, A2C, SAC


# env_id = "CartPole-v1"
# env = gym.make(env_id)

class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def step(self, action):
        obs, original_reward, terminated, truncated, info = self.env.step(action)
        
        # Extract angle and angular velocity from obs
        theta = obs[0]  # angle
        theta_dot = obs[1]  # angular velocity

        # Define your custom reward
        reward = - (theta**2 + 0.1 * theta_dot**2 + 0.01 * action**2)

        return obs, reward, terminated, truncated, info
    
    def reset(self, seed=None, options=None):
        if seed is not None:
            super().reset(seed=seed)
            self.np_random, _ = gym.utils.seeding.np_random(seed)
        
        # Reset my environment's state
        obs = super().reset()
        if isinstance(obs, tuple) and len(obs) == 2:
            obs = obs[0]
        # Return observation and empty info dict as expected by Gymnasium
        return obs, {}

base_env = gym_unbalanced_disk.UnbalancedDisk(dt=0.025, umax=3.)
env = CustomRewardWrapper(base_env)

# Fix compatibility with SB3
from gymnasium.wrappers import EnvCompatibility
env = EnvCompatibility(env)

# env = gym.make(env_id, render_mode='human')

# model = A2C('MlpPolicy', env, verbose=1)
model = PPO('MlpPolicy', env, verbose=1) #b)
model.learn(total_timesteps=25000)

obs = env.reset()[0]
try:
    for _ in range(1000):
        action, _states = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        env.render()
        # time.sleep(1/50)
        if terminated or truncated:
            env.reset()[0]
finally:
    env.close()

ImportError: cannot import name 'EnvCompatibility' from 'gymnasium.wrappers' (c:\Users\xskap\miniconda3\envs\ml4sc\Lib\site-packages\gymnasium\wrappers\__init__.py)

In [8]:
eval_env = gym_unbalanced_disk.UnbalancedDisk(umax=3.0, dt=0.025, render_mode='human')
num_eval_episodes = 5
for ep in range(num_eval_episodes):
    obs, _ = eval_env.reset()
    done = False
    total_reward = 0.0
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = eval_env.step(action)
        total_reward += reward
        eval_env.render()
        if terminated or truncated:
            done = True
    print(f"Episode {ep+1}: Total Reward = {total_reward:.3f}")

eval_env.close()

KeyboardInterrupt: 